<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Meta_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the model for MAML
class MAMLModel(nn.Module):
    def __init__(self):
        super(MAMLModel, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(10, 40),
            nn.ReLU(),
            nn.Linear(40, 1)
        )

    def forward(self, x):
        return self.fc(x)

# Function to simulate task-specific data
def generate_tasks(num_tasks, num_samples):
    tasks = []
    for _ in range(num_tasks):
        x_train = torch.randn(num_samples, 10)
        y_train = x_train.sum(dim=1, keepdim=True) + torch.randn(num_samples, 1) * 0.1  # Add noise
        x_val = torch.randn(num_samples, 10)
        y_val = x_val.sum(dim=1, keepdim=True) + torch.randn(num_samples, 1) * 0.1
        tasks.append((x_train, y_train, x_val, y_val))
    return tasks

# Meta-training step
def meta_train_step(model, optimizer, tasks, alpha=0.01, beta=0.001):
    meta_loss = 0.0
    for task_data in tasks:
        x_train, y_train, x_val, y_val = task_data

        # Create a copy of the model for task-specific adaptation
        model_copy = MAMLModel()
        model_copy.load_state_dict(model.state_dict())  # Clone global model

        # Inner loop: Task-specific learning
        inner_optimizer = optim.SGD(model_copy.parameters(), lr=alpha)
        for _ in range(5):  # Perform 5 inner-loop steps
            inner_optimizer.zero_grad()
            outputs = model_copy(x_train)
            loss = nn.MSELoss()(outputs, y_train)
            loss.backward()
            inner_optimizer.step()

        # Outer loop: Update global model
        optimizer.zero_grad()
        val_outputs = model_copy(x_val)
        val_loss = nn.MSELoss()(val_outputs, y_val)
        val_loss.backward()
        optimizer.step()

        meta_loss += val_loss.item()

    return meta_loss / len(tasks)

# Hyperparameters and setup
num_tasks = 10
num_samples_per_task = 20
tasks = generate_tasks(num_tasks, num_samples_per_task)
model = MAMLModel()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Meta-training loop
num_meta_epochs = 50
for epoch in range(num_meta_epochs):
    meta_loss = meta_train_step(model, optimizer, tasks)
    print(f"Epoch {epoch + 1}/{num_meta_epochs}, Meta Loss: {meta_loss:.4f}")